In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import nltk
import scipy
import warnings
import re

In [ ]:
df_train = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip',sep='\t')

df_test = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip',sep='\t')


In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
pd.options.display.max_colwidth = 500
print(df_train['review'])

In [ ]:
# Remove HTML tags
txt = " the source.<br /><br />Here's a pretensions:<br /><br />Just when"
cleanr = re.sub('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});','',txt)
cleanr

In [ ]:
# Removing digits
text = " the source.<br /><br />Here's a 20 preten 103s and 120th sions:<br /><br />Just when"
text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text) 
text

In [ ]:
# Let's clean text
def clean_text(text):
  text = text.lower()
  text = re.sub('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});','',text)  # for removal of html tags
  text = re.sub(r"can't","cannot",text)
  text = re.sub(r"shan't","shall not",text)
  text = re.sub(r"won't","will not",text)
  text = re.sub(r"n't"," not",text) # see the space before not. 
  text = re.sub(r"i'm","i am",text)
  text = re.sub(r"what's","what is",text)
  text = re.sub(r"let's","let us",text)
  text = re.sub(r"'re"," are",text)
  text = re.sub(r"'s"," ",text)  # space because we dont know the tense , it can be is/has anything.
  text = re.sub(r"'ve"," have",text)
  text = re.sub(r"\'ll", " will ", text)
  text = re.sub(r"\'scuse", " excuse ", text)
  text = re.sub(r"[^a-zA-Z]"," ",text)
  text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text) # Removing digits
  text = re.sub('\W', ' ', text)  # If the comment/word does not contain any alphabets
  text = re.sub('\s+', ' ', text) # If there are more than one whitespace simultenously, then replace them by only 1 whitespace and also replace the punctuation marks
  text = text.strip(' ') # Removing leading and trailing white spaces
  return text

In [ ]:
df_train['review'] = df_train['review'].apply(lambda text: clean_text(text))

In [ ]:
print(df_train['review'])

In [ ]:
df_test['review'] = df_test['review'].apply(lambda text: clean_text(text))

In [ ]:
print(df_test['review'])

In [ ]:
# For train file

X = df_train['review']
y = df_train['sentiment']

# For test file

X_test = df_test['review']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix,roc_auc_score,roc_curve

In [ ]:
tf = TfidfVectorizer(stop_words='english',max_features=50000)
tf

In [ ]:
# For training

X = tf.fit_transform(X)

# For test 

X_test = tf.transform(X_test)

In [ ]:
len(tf.get_feature_names()) ,

In [ ]:
print(X.toarray().shape)
X.toarray()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.1,random_state=50)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier()

In [ ]:
rf_clf.fit(X_train,y_train)

In [ ]:
y_val_pred = rf_clf.predict(X_val)

In [ ]:
print(accuracy_score(y_val_pred,y_val))

In [ ]:
print(roc_curve(y_val_pred,y_val))

In [ ]:
print(roc_auc_score(y_val_pred,y_val))

In [ ]:
rfc1 = rf_clf.predict(X_test)
rfc2 = rf_clf.predict_proba(X_test)[:,1]

In [ ]:
rfc1
rfc2

In [ ]:
submission_file = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv')
submission_file.head()

In [ ]:
submission_file['sentiment'] = rfc1

In [ ]:
submission_file

In [ ]:
submission_file.to_csv('Using ensemble model.csv',index=False)

~84% accuract

In [ ]:
import xgboost as xgb

In [ ]:
xg_clf = xgb.XGBClassifier(objective ='binary:logistic',learning_rate=0.2,n_estimators=1000,max_depth=20,tree_method = 'gpu_hist')

In [ ]:
xg_clf.fit(X_train,y_train)

In [ ]:
print(roc_auc_score(y_val_pred,y_val))

In [ ]:
xgb1 = xg_clf.predict(X_test)
xgb2 = xg_clf.predict_proba(X_test)[:,1]

In [ ]:
xgb2

In [ ]:
submission_file = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv')
submission_file.head()

In [ ]:
submission_file['sentiment'] = xgb1
submission_file.to_csv('Using xgboost.csv',index=False)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [ ]:
lr.fit(X_train,y_train)
y_val_pred = lr.predict(X_val)

print(roc_auc_score(y_val_pred,y_val))

lr1 = lr.predict(X_test)
lr2 = lr.predict_proba(X_test)[:,1]

In [ ]:
submission_file = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv')
submission_file['sentiment'] = lr1
submission_file.to_csv('Using Logistic Regression.csv',index=False)

In [ ]:
a = rfc2+xgb2+lr2
a = a/3
print(a)

In [ ]:
submission_file = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv')
submission_file['sentiment'] = a
submission_file.to_csv('Model Mixing.csv',index=False)

In [ ]:
a1 = xgb2+lr2
a1 = a1/2
print(a1)

In [ ]:
submission_file = pd.read_csv('/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv')
submission_file['sentiment'] = a1
submission_file.to_csv('2 Model Mixing.csv',index=False)